# Directories

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
vids = ["Video1_Clip1.mp4", "Video1_Clip2.mp4", "Video1_Clip3.mp4", "ShortClip1.mp4"]
weight_dirs = ["/kaggle/input/backup-tnv2/model906_30", "/kaggle/input/backup-tnv2/weights_k14_epoch19/weights_k14_epoch19/"]

***
# Imports and Helper Functions

In [ ]:
import sys
import getopt
import numpy as np
import os
from glob import glob
from keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.utils import array_to_img
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import *
from keras.layers import *
import keras.backend as K
from keras import optimizers
import tensorflow as tf
import cv2
from os.path import isfile, join
from PIL import Image
import time
import math

In [ ]:
from keras.models import *
from keras.layers import *
from keras.activations import *
def TrackNet3( input_height, input_width ): #input_height = 288, input_width = 512
	imgs_input = Input(shape=(9,input_height,input_width))
	#Layer1
	x = Conv2D(64, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(imgs_input)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer2
	x = Conv2D(64, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x1 = ( BatchNormalization())(x)
	#Layer3
	x = MaxPooling2D((2, 2), strides=(2, 2), data_format='channels_first' )(x1)
	#Layer4
	x = Conv2D(128, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer5
	x = Conv2D(128, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x2 = ( BatchNormalization())(x)
	#x2 = (Dropout(0.5))(x2) 
	#Layer6
	x = MaxPooling2D((2, 2), strides=(2, 2), data_format='channels_first' )(x2)
	#Layer7
	x = Conv2D(256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer8
	x = Conv2D(256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer9
	x = Conv2D(256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('relu'))(x)
	x3 = ( BatchNormalization())(x)
	#x3 = (Dropout(0.5))(x3)
	#Layer10
	x = MaxPooling2D((2, 2), strides=(2, 2), data_format='channels_first' )(x3)
	#Layer11
	x = ( Conv2D(512, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer12
	x = ( Conv2D(512, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer13
	x = ( Conv2D(512, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#x = (Dropout(0.5))(x)
	#Layer14
	#x = UpSampling2D( (2,2), data_format='channels_first')(x)
	x = concatenate( [UpSampling2D( (2,2), data_format='channels_first')(x), x3], axis=1)
	#Layer15
	x = ( Conv2D( 256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer16
	x = ( Conv2D( 256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer17
	x = ( Conv2D( 256, (3, 3), kernel_initializer='random_uniform', padding='same', data_format='channels_first'))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	
	#Layer18
	#x = UpSampling2D( (2,2), data_format='channels_first')(x)
	x = concatenate( [UpSampling2D( (2,2), data_format='channels_first')(x), x2], axis=1)
	#Layer19
	x = ( Conv2D( 128 , (3, 3), kernel_initializer='random_uniform', padding='same' , data_format='channels_first' ))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer20
	x = ( Conv2D( 128 , (3, 3), kernel_initializer='random_uniform', padding='same' , data_format='channels_first' ))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer21
	#x = UpSampling2D( (2,2), data_format='channels_first')(x)
	x = concatenate( [UpSampling2D( (2,2), data_format='channels_first')(x), x1], axis=1)
	#Layer22
	x = ( Conv2D( 64 , (3, 3), kernel_initializer='random_uniform', padding='same'  , data_format='channels_first' ))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer23
	x = ( Conv2D( 64 , (3, 3), kernel_initializer='random_uniform', padding='same'  , data_format='channels_first' ))(x)
	x = ( Activation('relu'))(x)
	x = ( BatchNormalization())(x)
	#Layer24
	x =  Conv2D( 3 , (1, 1) , kernel_initializer='random_uniform', padding='same', data_format='channels_first' )(x)
	x = ( Activation('sigmoid'))(x)
        
	o_shape = Model(imgs_input , x ).output_shape
	#print ("layer24 output shape:", o_shape[1],o_shape[2],o_shape[3])
	#Layer24 output shape: (3, 288, 512)
	OutputHeight = o_shape[2]
	OutputWidth = o_shape[3]
	output = x
	model = Model( imgs_input , output)
	#model input unit:9*288*512, output unit:3*288*512
	model.outputWidth = OutputWidth
	model.outputHeight = OutputHeight
	#Show model's details
	#model.summary()
	return model

In [ ]:
#Loss function
def custom_loss(y_true, y_pred):
	loss = (-1)*(K.square(1 - y_pred) * y_true * K.log(K.clip(y_pred, K.epsilon(), 1)) + K.square(y_pred) * (1 - y_true) * K.log(K.clip(1 - y_pred, K.epsilon(), 1)))
	return K.mean(loss)

In [ ]:
BATCH_SIZE=1
HEIGHT=288
WIDTH=512

***
# Load in old weights

In [ ]:
old_model = load_model(weight_dirs[0], custom_objects={'custom_loss':custom_loss})

In [ ]:
num_trainable_params = np.sum([K.count_params(w) for w in old_model.trainable_weights])
f"{num_trainable_params} trainable parameters"

In [ ]:
num_non_trainable_params = np.sum([K.count_params(w) for w in old_model.non_trainable_weights])
f"{num_non_trainable_params} non-trainable parameters"

***
# Load in new weights

In [ ]:
new_model = load_model(weight_dirs[1], custom_objects={'custom_loss':custom_loss})

In [ ]:
num_trainable_params = np.sum([K.count_params(w) for w in new_model.trainable_weights])
f"{num_trainable_params} trainable parameters"

In [ ]:
num_non_trainable_params = np.sum([K.count_params(w) for w in new_model.non_trainable_weights])
f"{num_non_trainable_params} non-trainable parameters"

***
# Prediction Function

In [ ]:
def make_pred_csv(predict_csv_dir, vid_dir, model):
    start = time.time()
    f = open(predict_csv_dir, 'w')
    f.write('Frame,Visibility,X,Y\n')
    cap = cv2.VideoCapture(vid_dir)
    cap.set(1,0) # set to 0'th frame?                            
    success, image1 = cap.read()
    success, image2 = cap.read()
    success, image3 = cap.read()
    ratio = image1.shape[0] / HEIGHT
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    count = 0
    
    start_time = time.time()
    while success:
        # count is in multiples of 3
        # report status every 99 frames
#         if count != 0 and (count/3) % 33 == 0:
#             end_time = time.time()
#             elapsed = round(end_time-start_time, 2) # seconds
#             estimated_total = round((num_frames/count)*(elapsed)/60/60,2) # hours
#             print(f"\t({count}/{num_frames} frames) {elapsed} seconds elapsed, {estimated_total} estimated hours total")

        unit = []
        #Adjust BGR format (cv2) to RGB format (PIL)
        x1 = image1[...,::-1]
        x2 = image2[...,::-1]
        x3 = image3[...,::-1]
        #Convert np arrays to PIL images
        x1 = array_to_img(x1)
        x2 = array_to_img(x2)
        x3 = array_to_img(x3)
        #Resize the images
        x1 = x1.resize(size = (WIDTH, HEIGHT))
        x2 = x2.resize(size = (WIDTH, HEIGHT))
        x3 = x3.resize(size = (WIDTH, HEIGHT))
        #Convert images to np arrays and adjust to channels first
        x1 = np.moveaxis(img_to_array(x1), -1, 0)
        x2 = np.moveaxis(img_to_array(x2), -1, 0)
        x3 = np.moveaxis(img_to_array(x3), -1, 0)
        #Create data
        unit.append(x1[0])
        unit.append(x1[1])
        unit.append(x1[2])
        unit.append(x2[0])
        unit.append(x2[1])
        unit.append(x2[2])
        unit.append(x3[0])
        unit.append(x3[1])
        unit.append(x3[2])
        unit=np.asarray(unit)	
        unit = unit.reshape((1, 9, HEIGHT, WIDTH))
        unit = unit.astype('float32')
        unit /= 255
        y_pred = model.predict(unit, batch_size=BATCH_SIZE, verbose=0)
        y_pred = y_pred > 0.5
        y_pred = y_pred.astype('float32')
        h_pred = y_pred[0]*255
        h_pred = h_pred.astype('uint8')
        for i in range(3):
            if i == 0:
                image = image1
            elif i == 1:
                image = image2
            elif i == 2:
                image = image3
            if np.amax(h_pred[i]) <= 0:
                f.write(str(count)+',0,-1,-1\n')
            else:
                #h_pred
                (cnts, _) = cv2.findContours(h_pred[i].copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                rects = [cv2.boundingRect(ctr) for ctr in cnts]
                max_area_idx = 0
                max_area = rects[max_area_idx][2] * rects[max_area_idx][3]
                for i in range(len(rects)):
                    area = rects[i][2] * rects[i][3]
                    if area > max_area:
                        max_area_idx = i
                        max_area = area
                target = rects[max_area_idx]
                (cx_pred, cy_pred) = (int(ratio*(target[0] + target[2] / 2)), int(ratio*(target[1] + target[3] / 2)))
                f.write(str(count)+',1,'+str(cx_pred)+','+str(cy_pred)+'\n')
            count += 1
            
        success, image1 = cap.read()
        success, image2 = cap.read()
        success, image3 = cap.read()
    f.close()
    end = time.time()
    print(f"\tVideo had {num_frames} frames. Predicted {count} frames. Last {num_frames - count} frames didn't get predicted.") 
    print(f"\tDone in {round(end-start, 2)} seconds")

***
# Run prediction function with old and new weights

In [ ]:
for vid in vids:
    vid_dir = f"/kaggle/input/backup-tnv2/{vid}"
    for version, model in [("old", old_model), ("new", new_model)]:
        predict_csv_dir = f"/kaggle/working/{vid[:-4]}_{version}_weights_predict.csv"
        print(f"Predicting {vid} with {version} weights...")
        make_pred_csv(predict_csv_dir, vid_dir, model)